In [ ]:
import pandas as pd
import seaborn as sns
from pyxlsb import open_workbook as open_xlsb
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [ ]:
from main import train_data

#EDA Analysis of numerical features
# Statistical indicators of numerical features
print("Statistical indicators of numerical features:")
numeric_stats = train_data.describe()
numeric_columns = numeric_stats.columns.drop("ID")
print(numeric_stats)

"""
# Boxlpot of numerical features
plt.figure(figsize=(10, 6))  # Размер графика (опционально)
train_data[numeric_columns].boxplot(grid=False)
plt.yscale("log")
plt.title("Boxplot числовых столбцов c логарифмической шкалой")
plt.ylabel("Значения")
plt.show()
"""

# Примените логарифмическое преобразование только к положительным значениям
train_data_log = train_data[numeric_columns].apply(lambda x: np.log1p(x))

# Постройте boxplot для логарифмированных числовых столбцов
plt.figure(figsize=(10, 6))  # Размер графика (опционально)
train_data_log.boxplot(grid=False)
plt.title("Boxplot логарифмированных числовых столбцов с помощью np.log1p")
plt.ylabel("Значения (логарифмическая шкала)")
plt.show()

# Постройте гистограммы распределения числовых признаков
plt.figure(figsize=(12, 8))  # Размер графика (опционально)
train_data[numeric_columns].hist(bins=20, color='skyblue', edgecolor='black', grid=False)
plt.suptitle("Гистограммы распределения числовых признаков")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# ToDo Выполнить после избавления от выбросов.
"""
# Определите все числовые признаки в данных
numeric_features = train_data[numeric_columns]

# Список возможных распределений для проверки
distributions = [
    'norm', 'expon', 'gamma', 'lognorm', 'beta',
    'pareto', 'uniform', 'triang', 'chi2', 'cauchy'
]

# Определите наилучшее соответствие распределения для каждого столбца
for col in numeric_columns:
    data = numeric_features[col].dropna()

    # Словарь для хранения результатов оценки соответствия распределений
    distribution_scores = {}

    # Попробуйте различные распределения и оцените их соответствие
    for dist_name in distributions:
        dist = getattr(stats, dist_name)
        params = dist.fit(data)
        _, ks_score = stats.kstest(data, dist_name, args=params)
        distribution_scores[dist_name] = ks_score

    # Найдите наиболее подходящее распределение по минимальному значению KS-статистики
    best_fit = min(distribution_scores, key=distribution_scores.get)

    print(f"Столбец '{col}' соответствует распределению: {best_fit}")
"""

# Вычислите матрицу корреляций для числовых признаков
correlation_matrix = train_data[numeric_columns].corr()

# Постройте тепловую карту (heatmap) для матрицы корреляций
plt.figure(figsize=(10, 8))  # Размер графика (опционально)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Матрица корреляций для числовых признаков")
plt.show()

def fit_distribution(data):
    """
    Определение наилучшего распределения данных с помощью критерия согласия Колмогорова-Смирнова.

    Parameters:
        data (pd.Series): Столбец данных.

    Returns:
        str: Наилучшее распределение, соответствующее данным.
    """
    # Список распределений для проверки
    distributions = ['norm', 'lognorm', 'expon', 'gamma', 'uniform']

    best_distribution = None
    best_params = ()
    best_kstest_stat = np.inf

    for distribution in distributions:
        # Получаем параметры распределения
        params = getattr(stats, distribution).fit(data, floc=0)

        # Вычисляем статистику критерия согласия Колмогорова-Смирнова
        kstest_stat, _ = stats.kstest(data, distribution, args=params)

        # Сохраняем параметры и статистику, если критерий согласия лучше
        if kstest_stat < best_kstest_stat:
            best_distribution = distribution
            best_params = params
            best_kstest_stat = kstest_stat

    return best_distribution, best_params

# Пример использования
# Замените 'dataframe' на ваш DataFrame, а 'numeric_columns' на список числовых столбцов
for column in numeric_columns:
   for column in numeric_columns:
    best_dist, best_params = fit_distribution(train_data[column])
    print(f"Столбец {column}: Наилучшее распределение - {best_dist}, Параметры - {best_params}")